In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import urllib.request

# All project files live here in your Google Drive
PROJECT_DIR = "/content/drive/MyDrive/phys2600_kinetic_project"
os.makedirs(PROJECT_DIR, exist_ok=True)

%cd $PROJECT_DIR

remote_dir = "https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/final_projects/proj3_kinetic/"
filenames = [
    "kinetic_API_tests.ipynb",
    "kinetic_assign.ipynb",
    "kinetic_presentation.ipynb",
    "kinetic.py",
]

for filename in filenames:
    remote_url = remote_dir + filename
    local_path = os.path.join(PROJECT_DIR, filename)
    if not os.path.exists(local_path):
        print(f"Copying {filename} to {local_path}")
        urllib.request.urlretrieve(remote_url, local_path)
    else:
        print(f"{filename} already exists at {local_path}")


# PHYS 2600 - Final Project: `kinetic`

## Statistical Mechanics of a Hard-Sphere Kinetic Gas

This is one possible _final project_; __you should choose one (and only one) project to complete and hand in.__  (If you try to work on two, I will grade the more complete one.  But these are big projects so I very strongly do NOT recommend attempting both!)

The deadline for this project is __7:00 PM (Boulder time) on Thursday, December 11.__ Since final grades must be submitted to the university shortly after the end of the exam period, to allow for timely grading _no late submissions will be accepted._

You are permitted to discuss the final projects with other students in the class,  including both the physics and computing aspects, but __your submitted project must be entirely your own__; no direct copying of Python code is permitted, and you must write up the presentation in your own words.

Your completed project will consist of __two (2) files__:

* `kinetic.py`: a Python module (see lecture 25) which implements the functions listed in the Application Progamming Interface (API) below.
* `kinetic_presentation.ipynb`: a Jupyter notebook which uses the code from your Python module to answer the physics questions.  Use of Markdown text, MathJax equations, and plots are all encouraged to make your presentation clear and compelling!


The rubric for grading will be split 50/50 between the code (in your `.py` module and Jupyter notebook) and the response to the physics questions (provided in `kinetic_presentation.ipynb`):
* Basic functionality of code (passes provided tests, follows API specification): __30%__
* __Six (6)__ additional API tests _you write_ (tests are provided and are correct and useful): __10%__
* Documentation of code (docstrings and comments make the code clear): __10%__
* Correct and detailed answers to physics questions: __40%__
* Quality of presentation (clear and readable, appropriate use of Markdown, equations, and plots): __10%__

A _bonus of up to 10%_ is available for the extra-credit physics question at the end.  (This challenge question is meant to be difficult!  But partial credit will be awarded if you make some progress on them.)

# Overview

Statistical mechanics allows us to make concrete predictions about the collective behavior of enormous numbers of objects, but the underlying models can be quite simple: the behavior of ideal gases can be derived by studying molecules which only undergo perfect elastic collisions and have no other interactions.  Computer simulation gives us a great opportunity to see the behavior predicted by statistical mechanics emerging explicitly!  

For this project, your task will be to build a kinematic simulation of a collection of elastic spheres with a non-zero radius (a __hard-sphere kinetic gas__.)  In the limit that the radius is small, you'll recover ideal gas behavior and other predictions of statistical mechanics.  With the radii set to be non-negligible, you'll begin to see deviations from the ideal gas law, corresponding to the onset of short-ranged van der Waals interactions.

# Physics background and important equations

For this project, we'll work in two spatial dimensions.  Three dimensions will be too taxing for the computer, and one dimension is too boring; two dimensions is enough to capture all of the important physics we're interested in.

## Statistical mechanics in two dimensions

Statistical mechanics relates the aggregate behavior of a collection of objects (here, gas molecules) to their microscopic properties.  The most famous result is the __ideal gas law__,

$$
PV = NkT.
$$

where $k$ is Boltzmann's constant, $k = 1.3807 \times 10^{-23}\ \rm{J/K}$.  This says that if we have a gas of $N$ molecules in a box of volume $V$, then the pressure $P$ and the temperature $T$ are related to each other.  

What are pressure and temperature, though?  Pressure is straightforward to define from a microscopic point of view; it's just the force exerted (by the gas molecules) per unit area on the walls of our box.  As for temperature, we know that it's related to how fast the molecules are moving.  Specifically, the _equipartition theorem_ relates the temperature of a system to the average kinetic energy of its constituents.  In a general number of spatial dimensions $d$, equipartition states that

$$
E = \frac{d}{2} kT
$$

Back to pressure; to understand how it arises, we have to start with the force.  If a molecule recoils elastically off of a wall (let's say in the $x$ direction), the change in momentum is

$$
\Delta p = 2mv_x.
$$

Now, the force exerted is $F = \Delta p / \Delta t$.  For any single molecule, the time between impacts in the $x$-direction will be $\Delta t = 2L/v_x$, the time it takes for the molecule to hit the opposite wall and come back.  So the force from a single particle is

$$
F = \frac{mv_x^2}{L}.
$$

There are $N$ particles flying around the box, and the area of the wall (assuming the box is cubic) is $L^{d-1}$, so we can put everything together to find

$$
P = \frac{Nm}{V} \langle v_x^2 \rangle
$$
where $\langle ... \rangle$ denotes the "thermodynamic average" over all the particles.

Now, we did this on a single wall in the $x$-direction, because (due to symmetry) the pressure on any wall will be the same.  However, we need to account for the fact that particles are moving in $d$ dimensions, which means that

$$
v^2 = \sum_{i=1}^d v_i^2.
$$

Once again, by symmetry the _average_ velocity components in any direction must be the same, so we can rewrite

$$
\langle v^2 \rangle = d \langle v_x^2 \rangle.
$$

The $1/d$ from this cancels out the $d$ from equipartition, and we end up with the ideal gas law - no matter what $d$ is!  So the ideal gas law is the same in two dimensions.  (More importantly, the pieces of this derivation will be useful when you want to calculate things like pressure and temperature from your simulation...)

The simplest modification we can make is to assume a __non-zero radius $R$__ for the gas molecules.  This change modifies the behavior away from ideal gas, leading to (part of) the __van der Waals equation__:

$$
\left(P + \frac{N^2 a}{V^2} \right) \left(V - Nb \right) = NkT
$$

The non-zero radius $R$ will lead to a non-zero $b$.  The other correction, $a$, only arises if we assume a long-distance _attractive_ force between the molecules, which we won't do for this simulation.



## Elastic collisions in two dimensions

There's only one bit of microscopic physics we need to understand to implement this project - elastic collisions of objects with a non-zero radius in two dimensions.  Dealing with walls is easy: a wall just __reverses the component of velocity perpendicular to the wall__.  (So a bounce off a wall at $x=+L$ takes the velocity from $(v_x, v_y)$ to $(-v_x, v_y)$, for example.)

To deal with two spheres (1 and 2) of non-zero radius colliding, we define a new vector called the __impact vector__,

$$
\mathbf{b} = \frac{\mathbf{x_2} - \mathbf{x_1}}{|\mathbf{x_2} - \mathbf{x_1}|} = \frac{\mathbf{x_2} - \mathbf{x_1}}{r_1 + r_2}
$$
Here $\mathbf{x_i}$ are the position vectors $(x,y)$ of the centers of the two spheres.  At the moment of collision, the distance between the centers $|\mathbf{x_2} - \mathbf{x_1}|$ is just the sum of the radii, $r_1 + r_2$.

The unit vector $\mathbf{b}$ points from the center of particle 1 to the center of particle 2.  We can define another unit vector

$$
\mathbf{c} = (-b_y, b_x)
$$
which is perpendicular to $\mathbf{b}$.  

This gives us a very convenient coordinate system: since the impact happens along the $b$ axis, we can treat it as a one-dimensional collision in that direction.  __For objects with equal mass__, an elastic one-dimensional collision just results in __exchange of velocity__.  So if the initial speeds are

$$
\mathbf{v_1} = v_{1b} \mathbf{b} + v_{1c} \mathbf{c} \\
\mathbf{v_2} = v_{2b} \mathbf{b} + v_{2c} \mathbf{c}
$$

then after the collision, we will have

$$
\mathbf{v_1}' = v_{2b} \mathbf{b} + v_{1c} \mathbf{c} \\
\mathbf{v_2}' = v_{1b} \mathbf{b} + v_{2c} \mathbf{c}.
$$

If the masses aren't equal, the final velocities are more complicated - see appendix A for the derivation and full formula.  (If you use the simpler formula, you should include an `assert` statement to verify that the masses of the spheres colliding are really equal!)

## Finding the exact time to collision

Our algorithm will be based on exact solutions (which are available to us with free-body motion!) coupled with checking for collisions.  To implement this, we need to solve for the _exact_ time of collision based on the current distance and velocity of two spheres.  (Walls are easy: once we know which wall, we just look along the $x$- or $y$-direction until the center of the sphere is within $r$ of the wall.)

To find exact time to collision between two particles 0 and 1 with radii $r_0$ and $r_1$ traveling on free-particle trajectories given by $\vec{x}_0(t)$ and $\vec{x}_1(t)$, we need to solve the equation
$$
|\vec{x}_0(t) - \vec{x}_1(t)|^2 = (r_0 + r_1)^2
$$
for the time of collision $t_c$.

Our particles travel in straight lines, so given initial position $\vec{x}_0 = (x_0, y_0)$ and velocity $\vec{v}_{0} = (v_{x0}, v_{y0})$, the trajectory of particle 0 is given by
$$
\vec{x}_0(t) = \vec{x}_0 + \vec{v}_0 t \\
    = (x_0 + v_{x0} t, y_0 + v_{y0} t)
$$
and similarly for particle 1, with $\vec{x}_1(t)$ defined in terms of with $\vec{x}_1$ and $\vec{v}_{1}$.

We can rewrite our collision condition as
$$
|\Delta \vec{x}(t)|^2 = \Delta x(t)^2 + \Delta y(t)^2 = (r_0 + r_1)^2 \equiv R^2
$$
where $\Delta \vec{x}(t) = \Delta \vec{x} + \Delta \vec{v}~t$ with
$$
\Delta \vec{x} = \vec{x}_1 - \vec{x}_0 = (x_1 - x_0, y_1 - y_0)
$$
and
$$
\Delta \vec{v} = \vec{v}_1 - \vec{v}_0 = (v_{x1} - v_{x0}, v_{y1} - v_{y0}).
$$
Plugging in to the quadratic equation, we find
$$
t = \frac{1}{|\Delta \vec{v}|^2}
    \left[
        - \Delta \vec{x} \cdot \Delta \vec{v}
        \pm
        \sqrt{
            (\Delta \vec{x} \cdot \Delta \vec{v})^2 -
            |\Delta \vec{v}|^2 (|\Delta \vec{x}|^2 - R^2)
        }
    \right]
$$
which is easily coded up in terms of the expressions
$$
    \Delta x = (x_1 - x_0) \\
    \Delta y = (y_1 - y_0) \\
    \Delta v_x = (v_{x1} - v_{x0}) \\
    \Delta v_y = (v_{y1} - v_{y0})
$$
so that we can write
$$
    |\Delta \vec{x}|^2 = \Delta x^2 + \Delta y^2,
$$
$$
    |\Delta \vec{v}|^2 = \Delta v_x^2 + \Delta v_y^2,
$$
and
$$
    \Delta \vec{x} \cdot \Delta \vec{v} = \Delta x \Delta v_x + \Delta y \Delta v_y.
$$

We find two solutions for $t$ in general.  To find the time of the next collision, we enforce that $t>0$ (i.e., we aren't looking for a collision in the past).  If both solutions $t_+$ and $t_-$ are positive, we keep the one that's less (i.e., we find the first solution).

# Computational Strategy and Algorithms

The starting point for our algorithm here is simple kinematics.  In fact, since there are no forces outside of collisions, we simply have the two equations

$$
\frac{d^2x}{dt^2} = \frac{d^2y}{dt^2} = 0.
$$

In terms of speed, we have that both $v_x$ and $v_y$ are constant, so we just need to write two simple non-zero-difference equations for position, updating $x$ and $y$ at each step as $(\Delta x, \Delta y) = (v_x dt, v_y dt)$.

### Dealing with collisions

The main complication is checking for wall collisions.  This is straightforward: after taking each step, we just check to see whether any part of the sphere is outside of one of the four walls of the box.  If we find a collision, we just reverse the velocity perpendicular to the wall.

Now, after finding a collision, we should __rewind the position__ of the sphere and let the motion continue from a point where it's _not_ overlapping a wall.  The simplest thing to do would be to just rewind to before we took the last step, but since the motion is so trivial, we can do better: we __rewind to the exact moment the collision takes place.__

So, our algorithm for collisions is the following:

* Update all positions by timestep $dt$.
* Look for any updated spheres that are overlapping or past a wall; these must have had a collision during the last $dt$.
* Calculate the exact time at which all collisions happened, between $0$ and $dt$ from the starting position.
* Find the time of the __first collision__ $dt'$ (the one that happens earliest.)
* Update the entire initial state using timestep $dt'$ instead of $dt$.  Now we're at the moment where a single collision happens.
* Resolve the collision: the velocity of the colliding sphere(s) changes according to the equations above, and everything else moves normally.

Although I started with a wall as an example, the algorithm is the same once we include collisions between spheres.  We always want to rewind and deal with the very first collision that happens during our proposed timestep, with a wall or with another sphere.

Note that this gives us a sequence of states _at the exact moment_ of every collision in the system.  We don't need to save anything else, because in between collisions, all of the motion is free - so we can easily interpolate.






# Application Programming Interface specification

__You must implement all of the below functions, according to the specifications given.__

A commonly-used construction below is the __sphere__, which is a tuple of the form

```
my_sphere = (m, r, x, y, vx, vy)
```
where `m` and `r` are the mass and radius of the sphere, `x` and `y` are the position, and `vx` and `vy` are the velocity.  Another important construction is the __state__, which is a list of all the spheres in our model.

Since we are working in four dimensions, we will denote the four walls of the box with cardinal directions `'N','S','E','W'`.  Our box is always a square, described by the length of a side `L`, and we fix the bottom left corner of the box to be at the origin $(x,y) = (0,0)$.  The equations for the walls are then

* `'N'`: Wall at $y=L$.
* `'E'`: Wall at $x=L$.
* `'S'`: Wall at $y=0$.
* `'W'`: Wall at $x=0$.

There is one major discretization error we need to watch out for: if our timestep `dt` is too large, two particles which should collide will just "jump over" each other, and we'll miss the collision completely.  So our algorithm begins with estimating a "safe timestep" to ensure that such collisions don't happen, based on the size of the spheres and how fast they're moving: if the smallest sphere has size $r_{\rm min}$, and the fastest sphere has speed $v_{\rm max}$, then

$$
dt < r_{\rm min}/(2 v_{\rm max})
$$

will be a safe timestep to use.  You should also check to make sure that no particles "jump over" one of the walls, which requires $dt < d_{\rm min} / (2 v_{\rm max})$, where $d_{\rm min}$ is the shortest distance from any particle to a wall.


### Evolution and finding collisions:

* `update_sphere_positions(state, dt)`: Given a state (list of sphere tuples) and a timestep `dt`, evolves the positions of all spheres into the future by `dt` without accounting for collisions (free motion.)
* `find_overlaps_in_state(state)`: Given a state, returns a list of _pairs of indices_, where each pair is a tuple of list indices corresponding to two different spheres which currently overlap.  Should only return __one tuple for each overlap__, i.e. if particles `state[0]` and `state[1]` overlap, it should return only `[(0,1)]` and not `[(0,1),(1,0)]`.
* `find_wall_collisions_in_state(state, L)`: Given a state, returns a list of pairs of spheres and walls which they are overlapping (or past), where each pair is a tuple containing the list index of a sphere and a cardinal direction (`NSEW`) indicating which wall it is overlapping/past.
* `compute_safe_timestep(state, L)`: Given a state, return a timestep `dt` which is 'small enough' that our simulation will be accurate. You may find it useful to find the maximum speed using `compute_speeds(state)` from above here.


### Handling collisions:

* `compute_time_to_sphere_collision(state, id0, id1)`: Given two sphere indices `id0` and `id1` (labeling their index within `state`), compute and return the time elapsed `dt` from their positions in `state` until they collide.
* `compute_time_to_wall_collision(state, id0, L, wall)`: Given a sphere index `id0` (labeling its index within `state`), the box size `L`, and a wall label (one of `N`/`S`/`E`/`W`), compute and return the time elapsed `dt` from the sphere's position in `state` until it collides with the given wall.
* `find_first_collision(state, sphere_hits, wall_hits, L)`: Given a (pre-collision!) state, two lists of expected collision tuples `sphere_hits` and `wall_hits` as returned by `find_overlaps_in_state` and `find_wall_collisions_in_state`, and the box size `L`, find and return `(collision_type, collision, dt)` corresponding to the first collision that happens (smallest time to collision.)  `collision_type` is a string which is either `'sphere'` or `'wall'`.
* `resolve_wall_collision(cur_state, id0, wall)`: Given the pre-collision state, a sphere index `id0`, and a wall label (one of `N`/`S`/`E`/`W`), returns an updated state where the velocity of the indicated sphere has been updated to account for its collision with the wall.
* `resolve_sphere_collision(p0, p1)`: Given two spheres `p0` and `p1` which are currently colliding (exactly in contact), returns an updated state where the sphere velocities have been updated to account for their collision. 


### Other functions:

* `random_initial_state(N_particles, L, v, r, m)`: Given the number of particles `N_particles`, box size `L`, initial speed `v`, sphere size `r`, and mass `m`, returns a state (list of spheres) corresponding to a valid initial state where all particles start with the same speed `v`, but random positions and moving in random directions.  "Valid" here means: no particles can be overlapping each other or the edges of the box.
* `get_state_at_time(t, times, states)`: From a completed simulation (the output of `simulate_gas` below), get the state at time `t` by interpolating between the known states and times.  (We found all the collisions, so any such interpolation is guaranteed to be just free motion.)
* `run_API_tests()`: A custom function that should use assertions to test the other functions implemented in the API.  If all tests are passed, the function should simply return the `None` object.  You should implement __at least six (6) tests__ inside this function on your own. I will provide additional tests.



## Main Loop (and special plotting code)

The below code implements the "main loop", running the simulation forward given an initial state `initial_state`, a time limit `max_time` to evolve for, and a box size `L`.  Once you have implemented the API above, add this code to your `kinetic.py` module, then call it to run the simulation!

In [ ]:
def simulate_gas(initial_state, max_time, L, verbose=False):
    """
    Main loop for hard-sphere kinetic gas project.

    Arguments:
    =====
    * initial_state: List of "sphere" tuples giving the initial state.
        A sphere tuple has the form:
            (m, r, x, y, vx, vy)
        where m is the mass, r is the radius, x and y are the coordinates, and
        vx and vy are the velocity.
    * max_time: The maximum amount of time to allow the system to evolve for.
        (Units depend on how you implement the API - be consistent!)
    * L: Side length of the square (LxL) box in which the simulation is run.
    * verbose: Flag to turn on "verbose mode" - prints diagnostic statements when true.
        (Warning: these print statements can greatly slow your code down, this flag
         should be False unless you're actively debugging!)

    Returns:
    =====
    (times, states): Two lists containing the times at which any collisions occurred,
        and the state of the system immediately after each collision.

    Example usage:
    =====
    >> state0 = random_initial_state(10, 200, 10, 2, 1)
    >> (times, states) = simulate_gas(state0, 60, 10)

    Now you can run measurements on `states`, or use the plot_state()
    function below to plot states after each collision, or even make an
    animation with get_state_at_time().

    """

    times = [0]
    states = [initial_state]
    eps = (
        1e-3  # "Overshoot" factor for moving past safe timestep - this should be small!
    )

    time = 0
    cur_state = initial_state

    while time < max_time:

        dt = compute_safe_timestep(cur_state, L)
        if verbose:
            print("Safe timestep = %g" % dt)

        # Try advancing state
        proposed = update_sphere_positions(cur_state, dt)

        # Check for wall or inter-sphere collisions
        sphere_hits = find_overlaps_in_state(proposed)
        wall_hits = find_wall_collisions_in_state(proposed, L)

        if verbose:
            print(" %d sphere hits, %d wall hits" % (len(sphere_hits), len(wall_hits)))

        if len(sphere_hits) == 0 and len(wall_hits) == 0:
            # Nothing interesting happened.  Keep state and move on
            time += dt
            cur_state = proposed
            if verbose:
                print("Nothing interesting happened at time", time)
            continue

        # Find first collision and what kind it was
        collision_type, first_collision, dt = find_first_collision(
            cur_state, sphere_hits, wall_hits, L
        )

        if verbose:
            print(
                "%s collision, (%s, %s), dt = %g"
                % (collision_type, first_collision[0], first_collision[1], dt)
            )

        # Handle collision
        proposed = update_sphere_positions(cur_state, dt * (1 + eps))

        if collision_type == "sphere":
            id0, id1 = first_collision

            # Resolve elastic collision and plug back in to state
            proposed = resolve_sphere_collision(proposed, id0, id1)

        elif collision_type == "wall":
            id0, wall0 = first_collision

            # Update state to account for wall collision
            proposed = bounce_particle_off_wall(proposed, id0, wall0)

        # Save new state and time
        time += dt
        cur_state = proposed
        times.append(time)
        states.append(cur_state)
        if verbose:
            print("Saving new state at time", time)

    # Save final state
    times.append(time)
    states.append(cur_state)
    return (times, states)

In addition, here's some special code for plotting a given state.  (We're providing plotting code in this project only, because plotting spheres with non-zero radius accurately turns out to be very arcane in matplotlib, especially if you want to animate them!)

In [ ]:
import matplotlib.patches as patches


def plot_state(state, color="blue"):
    # Extract positions
    r = []
    x = []
    y = []
    for m0, r0, x0, y0, vx0, vy0 in state:
        r.append(r0)
        x.append(x0)
        y.append(y0)

    # plt.scatter doesn't work for this -- scatter can't get size in data units, only "points"
    for nn in range(len(state)):
        plt.gca().add_artist(
            patches.Circle(xy=(x[nn], y[nn]), radius=r[nn], facecolor=color)
        )
        ax.set_aspect(1)

    return

And finally, an alternative version of this function to be used with `matplotlib.animation.ArtistAnimation` - see tutorial 19 and the API tests for how to set that up:

In [ ]:
def plot_state_frame(ax, state, color="blue"):
    # Extract positions
    r = []
    x = []
    y = []
    for m0, r0, x0, y0, vx0, vy0 in state:
        r.append(r0)
        x.append(x0)
        y.append(y0)

    # plt.scatter doesn't work for this -- scatter can't get size in data units, only "points"
    circles = []
    for nn in range(len(state)):
        circles.append(
            ax.add_artist(
                patches.Circle(xy=(x[nn], y[nn]), radius=r[nn], facecolor=color)
            )
        )
        ax.set_aspect(1)

    return circles

This would be used with `get_state_at_time` to make a smooth animation of your gas evolving in time, i.e.

```
fig, ax = plt.subplots()

ax.set_xlim(0,L)
ax.set_ylim(0,L)

frames = []
for t in np.linspace(0,10,50):
    frames.append(plot_state_frame(ax, get_state_at_time(t, times, states))
```

assuming that `times` and `states` contain the results of your simulation.  The list `frames` can then be used with `matplotlib.animation.ArtistAnimation` as you've seen before.

---
---
---

The below appendix is included for your information, but you don't need to understand it in gory detail for the project - just use the resulting formulas as given above.  (You may need it for the challenge question, though.)


## Appendix A: Elastic collisions in more detail

Particles 0 and 1 with radii $r_0$ and $r_1$ and masses $m_0$ and $m_1$ collide.  Instantaneously before the collision, their velocities are $\vec{v}_0$ and $\vec{v}_1$.  What are their velocities $\vec{v}_0'$ and $\vec{v}_1'$ instantaneously after the collision?

In any collision, momentum is conserved, so
$$
    m_0 \vec{v}_0 + m_1 \vec{v}_1 = m_0 \vec{v}_0' + m_1 \vec{v}_1'.
$$
The particles are perfectly hard spheres, which means their collisions are perfectly elastic and energy is also conserved and
$$
    \frac{1}{2} m_0 |\vec{v}_0|^2 + \frac{1}{2} m_1 |\vec{v}_1|^2
    = \frac{1}{2} m_0 |\vec{v}_0'|^2 + \frac{1}{2} m_1 |\vec{v}_1'|^2.
$$

At the moment of collision, the spheres at locations $\vec{x}_0$ and $\vec{x}_1$ are in contact, so we can constrain their positions.  They are separated by a distance of exactly $|\vec{x}_1 - \vec{x}_0| = r_0 + r_1$.  To set up a conveninent coordinate system, define the impact unit vector
$$
\hat{b} = \frac{\vec{x}_1 - \vec{x}_0}{|\vec{x}_1 - \vec{x}_0|} = \frac{\vec{x}_1 - \vec{x}_0}{r_1 + r_0}.
$$
The vector $\hat{b}$ points from the center of particle 0 to the center of particle 1. To get a coordinate system, define another unit vector perpendicular to $\hat{b}$,
$$
\hat{c} = (-b_y, b_x).
$$
We can now decompose our velocities in terms of components in the coordinate system defined by $\hat{b}$ and $\hat{c}$ like
$$
v_{0b} = \vec{v}_0 \cdot \hat{b} \\
v_{0c} = \vec{v}_0 \cdot \hat{c}
$$
and similarly for $v_{1b}$ and $v_{1c}$.

Because the particles are perfect spheres, and the forces they apply to each other are contact forces, they can only be applied perpendicular to the surface at the point of contact.  That means that the force on each particle is parallel with the impact vector $\hat{b}$.  Because the force is along $\hat{b}$, then the components of the velocities along the orthogonal direction $\hat{c}$ don't change during the collision, i.e.,
$$
v_{0c}' = v_{0c} \\
v_{1c}' = v_{1c}.
$$

We can write out our conservation of energy equation in terms of components of the velocity along $\hat{b}$ and $\hat{c}$ as
$$
    \frac{1}{2} m_0 v_{0b}^2 + \frac{1}{2} m_0 v_{0c}^2 +
    \frac{1}{2} m_1 v_{1b}^2 + \frac{1}{2} m_1 v_{1c}^2
    =
    \frac{1}{2} m_0 v_{0b}'^2 + \frac{1}{2} m_0 v_{0c}'^2 +
    \frac{1}{2} m_1 v_{1b}'^2 + \frac{1}{2} m_1 v_{1c}'^2.
$$
Because $v_{0c}' = v_{0c}$ and $v_{1c}' = v_{1c}$, the terms involving components in the $\hat{c}$ direction cancel, so we get an energy conservation equation just for the $\hat{b}$ components,
$$
    \frac{1}{2} m_0 v_{0b}^2
    \frac{1}{2} m_1 v_{1b}^2
    =
    \frac{1}{2} m_0 v_{0b}'^2
    \frac{1}{2} m_1 v_{1b}'^2.
$$
Momentum is also conserved separately in the $\hat{b}$ and $\hat{c}$ directions, so
$$
    m_0 v_{0b} + m_1 v_{1b} = m_0 v_{0b}' + m_1 v_{1b}'.
$$
Together, these two equations just look like a perfectly elastic collision in 1D, so we can reuse the textbook result for the final velocities in the $\hat{b}$ direction
$$
    v_{0b}' = \frac{m_0 - m_1}{m_0 + m_1} v_{0b} + \frac{2 m_1}{m_0 + m_1} v_{1b} \\
    v_{1b}' = \frac{2 m_0}{m_0 + m_1} v_{0b} - \frac{m_0 - m_1}{m_0 + m_1} v_{1b}.
$$
We then get the solutions in terms of our usual $\hat{x}$, $\hat{y}$ coordinate system as
$$
    \vec{v}_0' = v_{0b}' \hat{b} + v_{0c} \hat{c} \\
    \vec{v}_1' = v_{1b}' \hat{b} + v_{1c} \hat{c}.
$$